### 库

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from tqdm.notebook import tqdm, trange, tnrange
import json, re

In [2]:
import numpy as np
import pandas as pd
import os, sys
from time import time, sleep

In [3]:
from vVariableInspector import _getshapeof
from vUtil.vFile import fprint, readlines, linesReader
from vUtil.vEmail import sendEmail
from vUtil.vTime import convertSeconds, getNow

In [4]:
from vMysql import MysqlProxy
from elasticsearch import TransportError, ConnectionTimeout
import pymysql

In [5]:
from util import frmt, rmUnseen, groupConcat, deleteIndexRows 
from cfg import host, master, port, contType
from cfg import dbHost, dbUser, dbPwd
from cfg import ncols, nIdSep

### utils

In [6]:
def ourError (error, errorType = ''):
    fprint(f'[ERROR] ({getNow()}) {errorType} : {repr(error)}', file='error.txt', path='errorEnterprise')

def ourLog (log, logType = ''):
    fprint(f'[LOG] ({getNow()}) {logType} : {repr(log)}', file='log.txt', path='logEnterprise')

### mysql、es

In [7]:
index = 'landinn_enterprise'
indexHl = 'landinn_enterprise_highlight'

In [8]:
es=Elasticsearch(hosts=host,port=port,timeout=300)

In [9]:
db = MysqlProxy(ip=dbHost, user=dbUser, password=dbPwd)

### sql语句

In [10]:
sTmpEnterprise = '''
create temporary table tmp_enterprise
(
    select affiliation_id as enterprise_id from affiliations 
    where !ifnull(is_deleted,0)
    limit %d, %d
);
'''

sTmpEnterpriseId = '''
create temporary table tmp_enterprise
(
    select affiliation_id as enterprise_id from affiliations where affiliation_id = %d
);
'''

sRmTmpEnterprise = '''
drop temporary table tmp_enterprise;
'''

In [11]:
sTmpPublish = '''
create temporary table tmp_publish
(
    select affiliation_id as enterprise_id, sc_id as publish_id 
    from 
        softwareCopyright_affiliation as a
    join
        tmp_enterprise as b
    on a.affiliation_id = b.enterprise_id
    where !ifnull(a.is_deleted,0)
);'''

sInsertPublish = ['''
insert into tmp_publish 
(
    select applicant_id as enterprise_id, patent_id as publish_id 
    from 
        patent_applicants as a
    join
        tmp_enterprise as b
    on a.applicant_id = b.enterprise_id
    where !ifnull(a.is_deleted,0)
);
''',
'''
insert into tmp_publish 
(
    select affiliation_id as enterprise_id, golaxy_product_id as publish_id 
    from
        product as a
    join
        tmp_enterprise as b
    on a.affiliation_id = b.enterprise_id
    where !ifnull(a.is_deleted,0)
);
''']

sRmTmpPublish = '''
drop temporary table tmp_publish;
'''

In [12]:
def addPublishId (x, n = nIdSep):
    if n is None:
        s = "'曓攨爩氎廤攨攡擟戅嬼壣(',publish_id,')'"
    else: s = ','.join(["'(',publish_id,')'"] * n)
    return f'''concat({s},{x})'''

In [13]:
def getSqlSelectData (n = nIdSep):
    return f'''
    select  enterprise_id, 
            {addPublishId("softwareCopyright_name", n)} as softwareCopyright_name, 
            {addPublishId("simpleName", n)} as simpleName, 
            {addPublishId("patent_title", n)} as patent_title, 
            {addPublishId("signory", n)} as signory, 
            {addPublishId("summary", n)} as summary,
            {addPublishId("product_name", n)} as product_name, 
            {addPublishId("yewu", n)} as yewu, 
            {addPublishId("hangye", n)} as hangye
    from
    (
        select  enterprise_id, publish_id, softwareCopyright_name, simpleName, patent_title, 
                ifnull(patent_signory, patent_signory_en) as signory, 
                ifnull(patent_abstract, patent_abstract_en) as summary 
        from 
        (
            select enterprise_id, publish_id, softwareCopyright_name, simpleName, ifnull(patent_title, patent_title_en) as patent_title 
            from
            (
                select enterprise_id, publish_id, full_name as softwareCopyright_name, simple_name as simpleName
                from tmp_publish as a 
                left join software_copyright as b on a.publish_id = b.golaxy_sc_id
                where !ifnull(b.is_deleted,0)
            ) as c
            left join patent on c.publish_id = patent.golaxy_patent_id
            where !ifnull(patent.is_deleted,0)
        )as d
        left join patent_abstracts on d.publish_id = patent_abstracts.patent_id
        where !ifnull(patent_abstracts.is_deleted,0)
    ) as e
    left join product on e.publish_id = product.golaxy_product_id
    where !ifnull(product.is_deleted,0)
    ;
    '''

In [14]:
sSelectData = getSqlSelectData()
sSelectDataMini = getSqlSelectData(None) 

In [15]:
sSelectInfo = '''
select  affiliation_id as enterprise_id, 
        cast(not is_abroad as char) as is_chinese, 
        if(official_page is null, 0, 1) as has_officialPage,
        ifnull(display_name, display_name_en) as name, 
        industry,
        business_scope as businessScope,
        reg_province as province, 
        reg_city as city,
        cast(reg_capital_standard as char) as regCapital,
        establishment_time as establishmentTime
from tmp_enterprise as a
join affiliations as b
on a.enterprise_id = b.affiliation_id
where !ifnull(b.is_deleted,0)
;
'''

### 函数

In [16]:
def esRestart ():
    global es
    es.close()
    es=Elasticsearch(hosts=host,port=port,timeout=60)

In [17]:
def stdData (data):
    return {
        data['enterprise_id'][i] : {
            "softwareCopyright_name" : data['softwareCopyright_name'][i],
            "simpleName" : data['simpleName'][i],
            "patent_title" : data['patent_title'][i],
            "signory" : data['signory'][i],
            "summary" : data['summary'][i],
            "product_name" : data['product_name'][i],
            "yewu" : data['yewu'][i],
            "hangye" : data['hangye'][i],
        } for i in range(len(data))
    }

In [18]:
def stdInfo (info):
    return {
        info['enterprise_id'][i] : {
            'is_chinese' : info['is_chinese'][i],
            'has_officialPage' : info['has_officialPage'][i],
            'name' : info['name'][i],
            'industry' : info['industry'][i],
            'businessScope' : info['businessScope'][i],
            'province' : info['province'][i],
            'city' : info['city'][i],
            'regCapital' : info['regCapital'][i],
            'establishmentTime' : info['establishmentTime'][i],
        } for i in range(len(info))
    }

In [19]:
def getHighLightActions (data, info):
    actions = []
    for id in info:
        action={'_op_type':'index',###操作 index update create delete  
            '_index': indexHl,#index
            '_id' : str(id),
            '_source':
           {
                "enterprise_id" : str(id),
                "is_chinese" : info[id]['is_chinese'],
                "has_officialPage" : info[id]['has_officialPage'],
                "name" : info[id]['name'],
                "industry" : info[id]['industry'],
                "businessScope" : info[id]['businessScope'],
                "province" : info[id]['province'],
                "city" : info[id]['city'],
                "regCapital" : info[id]['regCapital'],
                "establishmentTime" : (None if info[id]['establishmentTime'] is pd.NaT 
                                       else info[id]['establishmentTime']),
            }
        }
        actions.append(action)
    return actions

In [20]:
def getActions (data, info):
    actions = []
    for id in info:
        action={'_op_type':'index',###操作 index update create delete  
            '_index': index,#index
            '_id' : str(id),
            '_source':
           {
                "enterprise_id" : str(id),
                "softwareCopyright_name" : data.get(id, {}).get('softwareCopyright_name', ''),
                "softwareCopyright_simpleName" : data.get(id, {}).get('simpleName', ''),
                "patent_title" : data.get(id, {}).get('patent_title', ''),
                "patent_signory" : data.get(id, {}).get('signory', ''),
                "patent_summary" : data.get(id, {}).get('summary', ''),
                "product_name" : data.get(id, {}).get('product_name', ''),
                "product_yewu" : data.get(id, {}).get('yewu', ''),
                "product_hangye" : data.get(id, {}).get('hangye', ''),
                "is_chinese" : info[id]['is_chinese'],
                "has_officialPage" : info[id]['has_officialPage'],
                "name" : info[id]['name'],
                "industry" : info[id]['industry'],
                "businessScope" : info[id]['businessScope'],
                "province" : info[id]['province'],
                "city" : info[id]['city'],
                "regCapital" : info[id]['regCapital'],
                "establishmentTime" : (None if info[id]['establishmentTime'] is pd.NaT 
                                       else info[id]['establishmentTime']),
            }
        }
        actions.append(action)
    return actions

In [21]:
def __getData (mode, *args):
    while 1:
        try:
            db.sql((sTmpEnterprise if mode else sTmpEnterpriseId) % args)
            db.sql(sTmpPublish)
            for x in sInsertPublish: db.sql(x)

            data = db.sql(sSelectData if mode else sSelectDataMini)

            data = groupConcat(data, 'enterprise_id')
            info = db.sql(sSelectInfo)
            db.sql(sRmTmpEnterprise)
            db.sql(sRmTmpPublish)
            db.close()
        except pymysql.Error as e:
            ourError(str(e), 'mysql read error')
            db.close()
            continue
        break
    return stdData(data), stdInfo(info)

In [22]:
def getRangeData (now, sizeBulk):
    return __getData(1, now, sizeBulk)
def getIdData (id):
    return __getData(0, id)

In [23]:
def miniInsert (data, info):
    for id in tqdm(info, leave=False):
        actions = getActions(data, {id : info[id]})
        while 1:
            try:
                helpers.bulk(client=es,actions=actions)
                break
            except ConnectionTimeout as e:
                ourError(str(e), f'connetion error with id({id})')
            except TransportError as e:
                if 'Data too large' in str(e) or e.status_code == 413:
                    ourError(str(e), f'Data too large with id({id})')
                    esRestart()

                    data, info = getIdData(id)
                    actions = getActions(data, info)
                    while 1:
                        try:
                            helpers.bulk(client=es,actions=actions)
                            break
                        except ConnectionTimeout as e:
                            ourError(str(e), f'connetion error mini indexing with id({id})')
                else: raise e

### 索引

In [24]:
table = 'affiliations'

In [25]:
deleteIndexRows(db, es, table, 'affiliation_id', index=[index,indexHl])

{'landinn_enterprise': 0, 'landinn_enterprise_highlight': 0}

In [25]:
sizeBulk = 20
now = 0
nEnterprise = db.count(table, where='!ifnull(is_deleted,0)').values.item()

In [26]:
# now = 49000

In [27]:
# sizeBulk = 50

In [29]:
now, sizeBulk, nEnterprise

(0, 20, 100705)

In [32]:
startOfAll = time()
tr = trange(now, nEnterprise, sizeBulk, ncols=ncols)
for i in tr:
    tr.set_description(f'({getNow()}){now}')
    now = i
    
    data, info = getRangeData(now, sizeBulk)

    while 1:
        try:
            helpers.bulk(client=es,actions=getHighLightActions(data,info))
            break
        except ConnectionTimeout as e:
            ourError(str(e), f'connetion error indexing hl with Range({now},{now + sizeBulk})')
    actions = getActions(data, info)
    while 1:
        try:
            helpers.bulk(client=es,actions=actions)
            break
        except ConnectionTimeout as e:
            ourError(str(e), f'connetion error with Range({now},{now + sizeBulk})')
        except TransportError as e:
            if 'Data too large' in str(e) or e.status_code == 413:
                ourError(str(e), f'Data too large with Range({now},{now + sizeBulk})')
                esRestart()
                miniInsert (data, info)
                break
            else: raise e
sendEmail(f'insert cost time {convertSeconds(time() - startOfAll)}', 'insert landinn enterprise es complete')

### 关闭es和mysql

In [34]:
es.close()

In [35]:
db.close()

### 检查是否全部索引

In [106]:
sizeBulk = 50
now = 0
nEnterprise = db.count('affiliations', where='is_deleted!=1').values.item()
for i in trange(now, nEnterprise, sizeBulk):
    now = i
        
    db.sql(sTmpEnterprise % (now, sizeBulk))

    info = db.sql(sSelectInfo)
    db.sql(sRmTmpEnterprise)
    db.close()
    
    info = stdInfo(info)
    for id in info:
        x = es.search(index=index, size=20, body = {
            "query": {
                "term":
                {
                    "enterprise_id": str(id)
                }
            }
        })['hits']['total']['value']
        if x == 0: print(now, id)

49000 2460
49000 88960587



### others

In [279]:
esRestart()

In [50]:
err = OperationalError()

In [73]:
paNotCom = r'(大学)|(学院)|(研究生院)|(学校)|(小学)|(中学)|(幼儿园)|'

In [96]:
paNotCom = r'大学|学院|研究生院|学校|小学|中学|幼儿园|'\
           r'高中|初中|一中$|二中$|三中$|'\
           r'研究所|研究院|研究中心|实验室|学会|创新中心'
re.search(paNotCom, '第一中')

In [99]:
x.group()

'一中'

In [ ]:
paCom = r'公司|企业|单位|厂$|医院'\
        r''\
        r''

In [82]:
re.search(paNotCom, '很好')